In [1]:
import pandas as pd
import numpy as np
persons = ["Daryn","Miras","Yerzhan","Person1"]
items = ["item1","item2","item3","item4"]
costs = {"item1":3000,"item2":2000,"item3":3500,"item4":1500}
links = {"Daryn":["item1","item3","item4"],"Miras":["item2","item4"],"Yerzhan":["item2"],"Person1":["item1", "item3"]}

In [2]:
dm = np.ndarray(shape = (len(items),len(persons)))
dm[0][1] = 1
print(dm)
for i in range(len(persons)):
    for j in range(len(items)):
        if items[j] in links[persons[i]]:
            dm[j][i] = 1
        else:
            dm[j][i] = 0
data = pd.DataFrame(data = dm, columns = persons)
data["sum"] = data.sum(axis = 1)
items = pd.Series(items)
data["items"] = items
data.head()

[[2.05833592e-312 1.00000000e+000 0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]]


,Daryn,Miras,Yerzhan,Person1,sum,items
0,1.0,0.0,0.0,1.0,2.0,item1
1,0.0,1.0,1.0,0.0,2.0,item2
2,1.0,0.0,0.0,1.0,2.0,item3
3,1.0,1.0,0.0,0.0,2.0,item4


In [3]:
costs = pd.DataFrame(costs,index = ["price"]).transpose()

costs.reset_index(inplace = True)
costs = costs.rename(columns = {'index':'items'})
costs.head()

,items,price
0,item1,3000
1,item2,2000
2,item3,3500
3,item4,1500


In [4]:
data = pd.merge(data, costs)
data.head()

,Daryn,Miras,Yerzhan,Person1,sum,items,price
0,1.0,0.0,0.0,1.0,2.0,item1,3000
1,0.0,1.0,1.0,0.0,2.0,item2,2000
2,1.0,0.0,0.0,1.0,2.0,item3,3500
3,1.0,1.0,0.0,0.0,2.0,item4,1500


In [5]:
# column_sum
for i in persons:
    data[i+"_total"] = data[i]/data["sum"]*data["price"]
data.head()

,Daryn,Miras,Yerzhan,Person1,sum,items,price,Daryn_total,Miras_total,Yerzhan_total,Person1_total
0,1.0,0.0,0.0,1.0,2.0,item1,3000,1500.0,0.0,0.0,1500.0
1,0.0,1.0,1.0,0.0,2.0,item2,2000,0.0,1000.0,1000.0,0.0
2,1.0,0.0,0.0,1.0,2.0,item3,3500,1750.0,0.0,0.0,1750.0
3,1.0,1.0,0.0,0.0,2.0,item4,1500,750.0,750.0,0.0,0.0


In [6]:
#Total Row
data.loc["Total_without_tip"] = data.sum()
data.head()
# leftover_columns =  
data = data.drop(persons + ["sum","items","price"],axis=1)
tip = 0.12
data.loc["Total"] = data.loc["Total_without_tip"]*(1+tip)
data.head(10)

,Daryn_total,Miras_total,Yerzhan_total,Person1_total
0,1500.0,0.0,0.0,1500.0
1,0.0,1000.0,1000.0,0.0
2,1750.0,0.0,0.0,1750.0
3,750.0,750.0,0.0,0.0
Total_without_tip,4000.0,1750.0,1000.0,3250.0
Total,4480.0,1960.0,1120.0,3640.0


In [7]:
# Mechanics of transactions
# 1. Pop the who pays guy and move to the end
who_pays = "Person1"
persons.remove(who_pays)
persons.append(who_pays)
print(persons)

# 2. Update totals
for i in range(len(persons)):
    if i > 0:
        data.loc["Total",persons[i]+"_total"] += data.loc["Total",persons[i-1]+"_total"]
data.head()

#3. Make Transactions
for i in range(len(persons)):
    if i == len(persons)-1:
        print("{} pays the bill".format(persons[i]))
    else:
        print("{} pays to {} {} tenge".format(persons[i],persons[i+1],data.loc["Total",persons[i]+"_total"]))

['Daryn', 'Miras', 'Yerzhan', 'Person1']
Daryn pays to Miras 4480.0 tenge
Miras pays to Yerzhan 6440.0 tenge
Yerzhan pays to Person1 7560.0 tenge
Person1 pays the bill
